# Main built-in LCEL Runnables

## Contents
* RunnablePassthrough.
* RunnableLambda.
* RunnableParallel.
    * itemgetter.
* RunnableBranch. 

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 005-builtin-runnables.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **005-builtin-runnables**.

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install python-dotenv

In [10]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [11]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

## RunnablePassthrough
* It does not do anything to the input data.
* Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

In [6]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [7]:
chain.invoke("Abram")

'Abram'

## RunnableLambda
* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
* Let's define a very simple function to create Russian lastnames:

In [3]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

* In order to use this function inside of a LCEL chain, we will use RunnableLambda():

In [8]:
from langchain_core.runnables import RunnableLambda

#chain = RunnablePassthrough() | RunnableLambda(russian_lastname)
chain = RunnableLambda(russian_lastname)

In [9]:
chain.invoke("Abram")

'Abramovich'

* As you see, our chain is now applying the russian_lastname function to our input.

## RunnableParallel
* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    * operation_a will use RunnablePassthrough.
    * operation_b will use RunnableLambda with the russian_lastname function.

In [10]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [11]:
chain.invoke("Abram")

{'operation_a': 'Abram', 'operation_b': 'Abramovich'}

* Instead of using RunnableLambda, now we are going to use a lambda function and we will invoke the chain with two inputs:

In [12]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": lambda x: x["name"]+"ovich"
    }
)

In [13]:
chain.invoke({
    "name1": "Jordan",
    "name": "Abram"
})

{'operation_a': {'name1': 'Jordan', 'name': 'Abram'},
 'soccer_player': 'Abramovich'}

* See how the lambda function is taking the "name" input.

#### We can add more Runnables to the chain
* In the following example, the prompt Runnable will take the output of the RunnableParallel:

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [15]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [16]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [17]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | model | output_parser

In [18]:
chain.invoke({
    "name1": "Jordan",
    "name": "Abram"
})

'A curious fact about Roman Abramovich is that he is not only known for his ownership of Chelsea Football Club but also for his diverse interests outside of football. In addition to being a prominent businessman and politician in Russia, Abramovich has an impressive collection of art, including works by renowned artists such as Claude Monet and Alberto Giacometti. His passion for art led him to establish the Garage Museum of Contemporary Art in Moscow, further showcasing his commitment to supporting the art community. This aspect of his life often goes unnoticed amid his high-profile football dealings.'

* As you saw, the prompt Runnable took "Abramovich", the output of the RunnableParallel, as the value for the "soccer_player" variable.

## Let's see a more advanced use of RunnableParallel

If you are using the pre-loaded poetry shell, you do not need to install the following packages because they are already pre-loaded for you:

In [ ]:
#!pip install faiss-cpu

In [ ]:
#!pip install langchain-community

In [19]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 10,000 Alumni from all continents and top companies"], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-4o-mini")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("who are the Alumni of AI Accelera?")

'The Alumni of AI Accelera are individuals from all continents and top companies who have been trained by the organization.'

#### Important: the syntax of RunnableParallel can have several variations.
* When composing a RunnableParallel with another Runnable you do not need to wrap it up in the RunnableParallel class. Inside a chain, the next three syntaxs are equivalent:
    * `RunnableParallel({"context": retriever, "question": RunnablePassthrough()})`
    * `RunnableParallel(context=retriever, question=RunnablePassthrough())`
    * `{"context": retriever, "question": RunnablePassthrough()}`

## Using itemgetter with RunnableParallel
* When you are calling the LLM with several different input variables.

In [6]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model = ChatOpenAI(model="gpt-4o-mini")

vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 5,000 Enterprise Alumni."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "How many Enterprise Alumni has trained AI Accelera?", "language": "Pirate English"})

'Arrr, matey! AI Accelera has trained over 5,000 Enterprise Alumni, savvy?'

## RunnableBranch: Router Chain
* A RunnableBranch is a special type of runnable that allows you to define a set of conditions and runnables to execute based on the input.
* **A RunnableBranch is initialized with a list of (condition, runnable) pairs and a default runnable**. It selects which branch by passing each condition the input it's invoked with. It selects the first condition to evaluate to True, and runs the corresponding runnable to that condition with the input.
* For advanced uses, a [custom function](https://python.langchain.com/v0.1/docs/expression_language/how_to/routing/) may be a better alternative than RunnableBranch.

The following advanced example can classify and respond to user questions based on specific topics like rock, politics, history, sports, or general inquiries. **It uses some new topics that we will explain in the following lesson**. Here’s a simplified explanation of each part:

1. **Prompt Templates**: Each template is tailored for a specific topic:
   - **rock_template**: Configured for rock and roll related questions.
   - **politics_template**: Tailored to answer questions about politics.
   - **history_template**: Designed for queries related to history.
   - **sports_template**: Set up to address sports-related questions.
   - **general_prompt**: A general template for queries that don't fit the specific categories.

   Each template includes a placeholder `{input}` where the actual user question will be inserted.

2. **RunnableBranch**: This is a branching mechanism that selects which template to use based on the topic of the question. It evaluates conditions (like `x["topic"] == "rock"`) to determine the topic and uses the appropriate prompt template.

3. **Topic Classifier**: A Pydantic class that classifies the topic of a user's question into one of the predefined categories (rock, politics, history, sports, or general).

4. **Classifier Chain**:
   - **Chain**: Processes the user's input to predict the topic.
   - **Parser**: Extracts the predicted topic from the classifier's output.

5. **RunnablePassthrough**: This component feeds the user's input and the classified topic into the RunnableBranch.

6. **Final Chain**:
   - The user's input is first processed to classify its topic.
   - The appropriate prompt is then selected based on the classified topic.
   - The selected prompt is used to formulate a question which is then sent to a model (like ChatOpenAI).
   - The model’s response is parsed as a string and returned.

7. **Execution**:
   - The chain is invoked with a sample question, "Who was Napoleon Bonaparte?" 
   - Based on the classification, it selects the appropriate template, generates a query to the chat model, and processes the response.

The system effectively creates a dynamic response generator that adjusts the way it answers based on the topic of the inquiry, making use of specialized knowledge for different subjects.

In [21]:
from langchain.prompts import PromptTemplate

rock_template = """You are a very smart rock and roll professor. \
You are great at answering questions about rock and roll in a concise\
and easy to understand manner.

Here is a question:
{input}"""

rock_prompt = PromptTemplate.from_template(rock_template)

politics_template = """You are a very good politics professor. \
You are great at answering politics questions..

Here is a question:
{input}"""

politics_prompt = PromptTemplate.from_template(politics_template)

history_template = """You are a very good history teacher. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods.

Here is a question:
{input}"""

history_prompt = PromptTemplate.from_template(history_template)

sports_template = """ You are a sports teacher.\
You are great at answering sports questions.

Here is a question:
{input}"""

sports_prompt = PromptTemplate.from_template(sports_template)

In [22]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch

In [23]:
general_prompt = PromptTemplate.from_template(
    "You are a helpful assistant. Answer the question as accurately as you can.\n\n{input}"
)
prompt_branch = RunnableBranch(
  (lambda x: x["topic"] == "rock", rock_prompt),
  (lambda x: x["topic"] == "politics", politics_prompt),
  (lambda x: x["topic"] == "history", history_prompt),
  (lambda x: x["topic"] == "sports", sports_prompt),
  general_prompt
)

In [18]:
from typing import Literal

from langchain.pydantic_v1 import BaseModel
from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from langchain_core.utils.function_calling import convert_to_openai_function

class TopicClassifier(BaseModel):
    "Classify the topic of the user question"
    
    topic: Literal["rock", "politics", "history", "sports"]
    "The topic of the user question. One of 'rock', 'politics', 'history', 'sports' or 'general'."

classifier_function = convert_to_openai_function(TopicClassifier)

llm = ChatOpenAI().bind(functions=[classifier_function], function_call={"name": "TopicClassifier"}) 

#parser = PydanticAttrOutputFunctionsParser(pydantic_schema=TopicClassifier, attr_name="topic")
from langchain.pydantic_v1 import BaseModel
from __main__ import TopicClassifier

print(issubclass(TopicClassifier, BaseModel))  # ✅ debe decir True
print(type(TopicClassifier))  # ✅ debe mostrar: <class 'langchain.pydantic_v1.main.ModelMetaclass'>

parser = PydanticAttrOutputFunctionsParser(pydantic_schema=TopicClassifier, attr_name="topic")

classifier_chain = llm | parser

True
<class 'pydantic.v1.main.ModelMetaclass'>


ValidationError: 2 validation errors for PydanticAttrOutputFunctionsParser
pydantic_schema.is-subclass[BaseModel]
  Input should be a subclass of BaseModel [type=is_subclass_of, input_value=<class '__main__.TopicClassifier'>, input_type=ModelMetaclass]
    For further information visit https://errors.pydantic.dev/2.11/v/is_subclass_of
pydantic_schema.dict[str,is-subclass[BaseModel]]
  Input should be a valid dictionary [type=dict_type, input_value=<class '__main__.TopicClassifier'>, input_type=ModelMetaclass]
    For further information visit https://errors.pydantic.dev/2.11/v/dict_type

In [17]:
from typing import Literal
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

# ✅ Modelo Pydantic v2
class TopicClassifier(BaseModel):
    """Classify the topic of the user question"""
    topic: Literal["rock", "politics", "history", "sports"]

# ✅ Conversión a función de OpenAI
classifier_function = convert_to_openai_function(TopicClassifier)

# ✅ Modelo LLM configurado para usar la función
llm = ChatOpenAI().bind(
    functions=[classifier_function],
    function_call={"name": "TopicClassifier"}
)

# ✅ Ejecución
response = llm.invoke("Tell me about the Cold War.")

# ✅ Extracción manual del resultado
args = response.additional_kwargs.get("function_call", {}).get("arguments")
import json
parsed_args = json.loads(args)
result = TopicClassifier.model_validate(parsed_args)

print(result.topic)  # Output esperado: "history"


history


The `classifier_function` classifies or categorizes the topic of a user's question into specific categories such as "rock," "politics," "history," or "sports." Here’s how it works in simple terms:

1. **Conversion to Function**: It converts the `TopicClassifier` Pydantic class, which is a predefined classification system, into a function that can be easily used with LangChain. This conversion process involves wrapping the class so that it can be integrated and executed within an OpenAI model.

2. **Topic Detection**: When you input a question, this function analyzes the content of the question to determine which category or topic it belongs to. It looks for keywords or patterns that match specific topics. For example, if the question is about a rock band, the classifier would identify the topic as "rock."

3. **Output**: The function outputs the identified topic as a simple label, like "rock" or "history." This label is then used by other parts of the LangChain to decide how to handle the question, such as choosing the right template for formulating a response.

In essence, the `classifier_function` acts as a smart filter that helps the system understand what kind of question is being asked so that it can respond more accurately and relevantly.

In [26]:
from operator import itemgetter

from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


final_chain = (
    RunnablePassthrough.assign(topic=itemgetter("input") | classifier_chain) 
    | prompt_branch 
    | ChatOpenAI()
    | StrOutputParser()
)

In [38]:
final_chain.invoke(
    {"input":"Who is Matei Zaharia " }
)

'Matei Zaharia is a Romanian-Canadian computer scientist and co-founder of Databricks, a company that provides a unified analytics platform. He is also the creator of Apache Spark, an open-source cluster computing framework. Zaharia is known for his work in big data processing and machine learning, and has made significant contributions to the field of distributed computing.'

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 005-builtin-runnables.py
* In terminal, make sure you are in the directory of the file and run:
    * python 005-builtin-runnables.py